In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "./qwen-sft-final"  # your finetuned checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",          # use GPU if available
    torch_dtype=torch.float16   # or float32 if CPU
)

prompt = "What is lung cancer?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,          # optional: add sampling
        top_p=0.9,
        temperature=0.7
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


`torch_dtype` is deprecated! Use `dtype` instead!
2025-10-05 09:59:43.011211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-05 09:59:43.011237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-05 09:59:43.011970: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-05 09:59:43.015134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")

model = AutoModelForCausalLM.from_pretrained(
    "./qwen-sft-final",
    device_map="auto",         # automatically use GPU if available
    torch_dtype="auto"         # safe precision
)

prompt = "What is lung cancer?"
inputs = tokenizer(prompt, return_tensors="pt")

# match device
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    output_ids = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


2025-10-05 11:32:46.892358: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-05 11:32:46.892383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-05 11:32:46.893133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-05 11:32:46.897007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-05 11:32:47.420885: W tensorflow/compiler/tf2

What is lung cancer?!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, "./qwen2-0.5B-lora-finetuned")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B", use_fast=True)



`torch_dtype` is deprecated! Use `dtype` instead!


In [5]:
from transformers import pipeline

# Do NOT pass device=0
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

question = "Question: What are the common symptoms of lung cancer?\nAnswer:"
output = pipe(question, max_new_tokens=100)
print(output[0]['generated_text'])



Device set to use cuda:0


Question: What are the common symptoms of lung cancer?
Answer: A family history of lung cancer typically runs around 5-6 generations.

